## Data Preprocessing
Prior to the start of the project, it is essential to undertake a data pre-processing phase in order to improve the quality of the data to be used in the subsequent development phases of the player Scouting recommendation system. During this phase, a series of data cleaning and preparation tasks are performed to ensure that the data are consistent and ready for further processing. This includes identifying and correcting any errors, handling missing data, removing redundant information and standardising data formats. The ultimate goal is to obtain a consistent and reliable set of data that can be used effectively in the subsequent phases of the project. 

In [1]:
# used libraries 
import pandas as pd
import datetime
from sklearn.feature_selection import VarianceThreshold

### Data Retrieval
The initial phase involved retrieving data from the Football Reference portal (FBRef). 
Data were collected using the 'Extract from web site' functionality of Excel. The data represent the statistics of a specific football player according to his position (e.g. goalkeeper, striker). This data was then imported into pandas DataFrames for further processing. To ensure data consistency, all rows containing irrelevant data were removed from consideration. This initial step ensured the acquisition of structured raw data.

In [2]:
# Retrieve raw data and Clear

# Complete data with stats complete of football players
df1 = pd.read_csv('23-24/FootballPlayerStats_COMPLETE_202324.csv', sep=',')  # Try with commas
df1.columns = df1.columns.str.strip()
print(df1.columns)

df1 = df1[df1['Rk'] != 'Rk']
df1.drop('Matches', axis=1, inplace=True)

df2 = pd.read_csv('23-24/FootballPlayerStats_DF_202324.csv', sep=',')
df2 = df2[df2['Rk'] != 'Rk']
df2.drop('Matches', axis=1, inplace=True)

df3 = pd.read_csv('23-24/FootballPlayerStats_FW_202324.csv', sep=',')
df3 = df3[df3['Rk'] != 'Rk']
df3.drop('Matches', axis=1, inplace=True)

df4 = pd.read_csv('23-24/FootballPlayerStats_GK_202324.csv', sep=',')
df4 = df4[df4['Rk'] != 'Rk']
df4.drop('Matches', axis=1, inplace=True)

df5 = pd.read_csv('23-24/FootballPlayerStats_MF_202324.csv', sep=',')
df5 = df5[df5['Rk'] != 'Rk']
df5.drop('Matches', axis=1, inplace=True)

df6 = pd.read_csv('23-24/FootballPlayerStats_MISC_202324.csv', sep=',')
df6 = df6[df6['Rk'] != 'Rk']
df6.drop('Matches', axis=1, inplace=True)

df7 = pd.read_csv('23-24/FootballPlayerStats_PASS_202324.csv', sep=',')
df7 = df7[df7['Rk'] != 'Rk']
df7.drop('Matches', axis=1, inplace=True)



Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born',
       'Playing Time MP', 'Playing Time Starts', 'Playing Time Min',
       'Playing Time 90s', 'Performance Gls', 'Performance Ast',
       'Performance G+A', 'Performance G-PK', 'Performance PK',
       'Performance PKatt', 'Performance CrdY', 'Performance CrdR',
       'Expected xG', 'Expected npxG', 'Expected xAG', 'Expected npxG+xAG',
       'Progression PrgC', 'Progression PrgP', 'Progression PrgR',
       'Per 90 Minutes Gls', 'Per 90 Minutes Ast', 'Per 90 Minutes G+A',
       'Per 90 Minutes G-PK', 'Per 90 Minutes G+A-PK', 'Per 90 Minutes xG',
       'Per 90 Minutes xAG', 'Per 90 Minutes xG+xAG', 'Per 90 Minutes npxG',
       'Per 90 Minutes npxG+xAG', 'Matches'],
      dtype='object')


In [3]:
# delete potentially duplicate columns

df6.drop(['Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', '90s','Performance CrdY', 'Performance CrdR', 'Performance TklW',
       'Performance PKwon', 'Performance PKcon'],axis=1,inplace=True)
df6.columns

df7.drop(['Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', '90s','Short Cmp', 'Short Att', 'Short Cmp%', 'Medium Cmp',
       'Medium Att', 'Medium Cmp%', 'Long Cmp', 'Long Att', 'Long Cmp%', ],axis=1,inplace=True)
df7.columns

Index(['Rk', 'Player', 'Total Cmp', 'Total Att', 'Total Cmp%', 'Total TotDist',
       'Total PrgDist', 'Ast', 'xAG', 'xA', 'A-xAG', 'KP', '1/3', 'PPA',
       'CrsPA', 'PrgP'],
      dtype='object')

In [4]:
# Merge the renamed dataframes using the 'Player' column as the merge key
# First merge with df2 and df3
combined_df = pd.merge(df2, df3, on='Rk', how='outer', suffixes=('_df2', '_df3'))

# Then merge with df5
combined_df = pd.merge(combined_df, df5, on='Rk', how='outer', suffixes=('', '_df5'))

# Then merge with df6
combined_df = pd.merge(combined_df, df6, on='Rk', how='outer', suffixes=('', '_df6'))

# Finally, merge with df7
combined_df = pd.merge(combined_df, df7, on='Rk', how='outer', suffixes=('', '_df7'))


### Data cleaning 
In the data cleaning phase, rigorous quality control measures were applied. Duplicate rows were identified and removed to prevent redundancy, ensuring the dataset remained concise. Additionally, any missing or incomplete data entries, often denoted as NaN (Not a Number), were addressed using appropriate techniques. To maintain uniformity and facilitate subsequent computations, data type conversions were performed, ensuring all values adhered to a consistent numeric format.

In [5]:
# Merge dataset with Golkeeping 
combined_df = pd.merge(combined_df, df4, left_on='Player', right_on='Player', how='outer')

In [6]:
# Select only unduplicated columns
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]
#len(combined_df.columns)

In [7]:
# Checkpoint 
combined_df

,Rk_x,Player_df2,Nation_df2,Pos_df2,Squad_df2,Comp_df2,Age_df2,Born_df2,90s_df2,Tackles Tkl,...,Performance W,Performance D,Performance L,Performance CS,Performance CS%,Penalty Kicks PKatt,Penalty Kicks PKA,Penalty Kicks PKsv,Penalty Kicks PKm,Penalty Kicks Save%
0,1,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,23,2000,13.7,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Brenden Aaronson,us USA,"MF,FW",Union Berlin,de Bundesliga,22,2000,14.1,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Paxten Aaronson,us USA,MF,Eint Frankfurt,de Bundesliga,19,2003,1.1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,17,2006,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,35,1987,30.9,64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849,2848,Lovro Zvonarek,hr CRO,"FW,MF",Bayern Munich,de Bundesliga,18,2005,1.8,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2850,2849,Martin Ødegaard,no NOR,MF,Arsenal,eng Premier League,24,1998,34.3,49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2851,2850,Milan Đurić,ba BIH,FW,Hellas Verona,it Serie A,33,1990,13.4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2852,2851,Milan Đurić,ba BIH,FW,Monza,it Serie A,33,1990,14,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Rename
column_mapping = {
    'Rk_x': 'Rk',
    'Player_x': 'Player',
    'Nation_x': 'Nation',
    'Pos_x': 'Pos',
    'Squad_x': 'Squad',
    'Comp_x': 'Comp',
    'Age_x': 'Age',
    'Born_x': 'Born',
    '90s_x': '90s'
}

# Rename the columns using the dictionary
combined_df = combined_df.rename(columns=column_mapping)

# Delete column respect to Player and Squad_x
combined_df = combined_df.drop_duplicates(subset=['Player','Squad'])

# Delete column specified
columns_to_drop = ['Nation_y', 'Pos_y', 'Squad_y', 'Comp_y', 'Age_y', 'Born_y', '90s_y']
combined_df = combined_df.drop(columns=columns_to_drop)

# Fill NaN with 0
combined_df = combined_df.fillna(0)

In [9]:
# Checkpoint
combined_df

,Rk,Player_df2,Nation_df2,Pos_df2,Squad_df2,Comp_df2,Age_df2,Born_df2,90s_df2,Tackles Tkl,...,Performance W,Performance D,Performance L,Performance CS,Performance CS%,Penalty Kicks PKatt,Penalty Kicks PKA,Penalty Kicks PKsv,Penalty Kicks PKm,Penalty Kicks Save%
0,1,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,23,2000,13.7,29,...,0,0,0,0,0,0,0,0,0,0
1,2,Brenden Aaronson,us USA,"MF,FW",Union Berlin,de Bundesliga,22,2000,14.1,32,...,0,0,0,0,0,0,0,0,0,0
2,3,Paxten Aaronson,us USA,MF,Eint Frankfurt,de Bundesliga,19,2003,1.1,2,...,0,0,0,0,0,0,0,0,0,0
3,4,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,17,2006,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,35,1987,30.9,64,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849,2848,Lovro Zvonarek,hr CRO,"FW,MF",Bayern Munich,de Bundesliga,18,2005,1.8,0,...,0,0,0,0,0,0,0,0,0,0
2850,2849,Martin Ødegaard,no NOR,MF,Arsenal,eng Premier League,24,1998,34.3,49,...,0,0,0,0,0,0,0,0,0,0
2851,2850,Milan Đurić,ba BIH,FW,Hellas Verona,it Serie A,33,1990,13.4,1,...,0,0,0,0,0,0,0,0,0,0
2852,2851,Milan Đurić,ba BIH,FW,Monza,it Serie A,33,1990,14,5,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Merge dataset with Standard Data Players
# If 'Rk_x' or 'Rk_y' exists, rename them
combined_df.rename(columns={'Rk_x': 'Rk', 'Rk_y': 'Rk'}, inplace=True)
df1.rename(columns={'Rk_x': 'Rk', 'Rk_y': 'Rk'}, inplace=True)


# Drop duplicate 'Rk' columns, keeping the first occurrence
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]
df1 = df1.loc[:, ~df1.columns.duplicated()]

# Merge dataset with Standard Data Players
merged_df = pd.merge(combined_df, df1, on='Rk', how='outer')
merged_df = merged_df.fillna(0)


In [11]:
# Fill in the missing values in the following columns with 0 (as integer)
merged_df['Playing Time Min_x'] = merged_df['Playing Time Min_x'].fillna(0)
merged_df['Playing Time Min_y'] = merged_df['Playing Time Min_y'].fillna(0)

# Removing commas (,) from columns
merged_df['Playing Time Min_x'] = merged_df['Playing Time Min_x'].astype(str).str.replace(',', '')
merged_df['Playing Time Min_y'] = merged_df['Playing Time Min_y'].astype(str).str.replace(',', '')

# Convert columns to integer (int) data type
merged_df['Playing Time Min_x'] = merged_df['Playing Time Min_x'].astype(int)
merged_df['Playing Time Min_y'] = merged_df['Playing Time Min_y'].astype(int)


In [12]:
# Check existing columns before dropping
print("Current columns in merged_df:", merged_df.columns.tolist())

# Define columns to drop, checking if they exist
columns_to_drop = ['Rk_y', 'Nation_y', 'Pos_y', 'Squad_y', 'Comp_y', 'Age_y', 'Born_y', 
                   'Playing Time MP_y', 'Playing Time Starts_y', 'Playing Time Min_y', 
                   'Expected xG_y', 'Expected npxG_y']

# Drop columns safely, ignoring errors for non-existent columns
merged_df = merged_df.drop(columns=columns_to_drop, errors='ignore')

# Fill NaN data with 0
merged_df = merged_df.fillna(0)

# Show remaining columns
print("Remaining columns after dropping:", merged_df.columns.tolist())


Current columns in merged_df: ['Rk', 'Player_df2', 'Nation_df2', 'Pos_df2', 'Squad_df2', 'Comp_df2', 'Age_df2', 'Born_df2', '90s_df2', 'Tackles Tkl', 'Tackles TklW', 'Tackles Def 3rd', 'Tackles Mid 3rd', 'Tackles Att 3rd', 'Challenges Tkl', 'Challenges Att', 'Challenges Tkl%', 'Challenges Lost', 'Blocks Blocks', 'Blocks Sh', 'Blocks Pass', 'Int', 'Tkl+Int', 'Clr', 'Err', 'Player_df3', 'Nation_df3', 'Pos_df3', 'Squad_df3', 'Comp_df3', 'Age_df3', 'Born_df3', '90s_df3', 'Standard Gls', 'Standard Sh', 'Standard SoT', 'Standard SoT%', 'Standard Sh/90', 'Standard SoT/90', 'Standard G/Sh', 'Standard G/SoT', 'Standard Dist', 'Standard FK', 'Standard PK', 'Standard PKatt', 'Expected xG_x', 'Expected npxG_x', 'Expected npxG/Sh', 'Expected G-xG', 'Expected np:G-xG', 'Player_x', 'Nation_x', 'Pos_x', 'Squad_x', 'Comp_x', 'Age_x', 'Born_x', '90s', 'SCA SCA', 'SCA SCA90', 'SCA Types PassLive', 'SCA Types PassDead', 'SCA Types TO', 'SCA Types Sh', 'SCA Types Fld', 'SCA Types Def', 'GCA GCA', 'GCA GCA9

In [13]:
# Rename columns
column_mapping = {
    'Rk_x': 'Rk',
    'Player_x': 'Player',
    'Nation_x': 'Nation',
    'Pos_x': 'Pos',
    'Squad_x': 'Squad',
    'Comp_x': 'Comp',
    'Age_x': 'Age',
    'Born_x': 'Born',
    '90s_x': '90s',
    'Expected xG_x': 'Expected xG',
    'Expected npxG_x': 'Expected npxG',
    'Playing Time MP_x': 'Playing Time MP',
    'Playing Time Starts_x': 'Playing Time Starts',
    'Playing Time Min_x': 'Playing Time Min',
    'xA': 'Expected xA',
    'A-xAG': 'Expected A-xAG'

}

# Rename the columns using the dictionary
merged_df = merged_df.rename(columns=column_mapping)


In [15]:
# Checkpoint
merged_df

,Rk,Player_df2,Nation_df2,Pos_df2,Squad_df2,Comp_df2,Age_df2,Born_df2,90s_df2,Tackles Tkl,...,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes G+A,Per 90 Minutes G-PK,Per 90 Minutes G+A-PK,Per 90 Minutes xG,Per 90 Minutes xAG,Per 90 Minutes xG+xAG,Per 90 Minutes npxG,Per 90 Minutes npxG+xAG
0,1,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,23,2000,13.7,29,...,0,0.07,0.07,0,0.07,0,0.06,0.06,0,0.06
1,2,Brenden Aaronson,us USA,"MF,FW",Union Berlin,de Bundesliga,22,2000,14.1,32,...,0.14,0.14,0.28,0.14,0.28,0.14,0.13,0.27,0.14,0.27
2,3,Paxten Aaronson,us USA,MF,Eint Frankfurt,de Bundesliga,19,2003,1.1,2,...,0,0.89,0.89,0,0.89,0.11,0.07,0.19,0.11,0.19
3,4,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,17,2006,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,35,1987,30.9,64,...,0.13,0,0.13,0.1,0.1,0.11,0.01,0.12,0.09,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2847,2848,Lovro Zvonarek,hr CRO,"FW,MF",Bayern Munich,de Bundesliga,18,2005,1.8,0,...,0.55,0,0.55,0.55,0.55,0.04,0.12,0.15,0.04,0.15
2848,2849,Martin Ødegaard,no NOR,MF,Arsenal,eng Premier League,24,1998,34.3,49,...,0.23,0.29,0.52,0.17,0.47,0.22,0.28,0.5,0.17,0.45
2849,2850,Milan Đurić,ba BIH,FW,Hellas Verona,it Serie A,33,1990,13.4,1,...,0.37,0.07,0.45,0.3,0.37,0.34,0.09,0.43,0.16,0.25
2850,2851,Milan Đurić,ba BIH,FW,Monza,it Serie A,33,1990,14,5,...,0.29,0.07,0.36,0.29,0.36,0.21,0.05,0.26,0.21,0.26


In [16]:
# Calculate the current year
merged_df['Born'] = merged_df['Born'].astype(int)
current_year = datetime.datetime.now().year

# Calculates age by subtracting the year of birth from the current date
merged_df['Age'] = current_year - merged_df['Born']

In [17]:
# Reorder columns based on similar meanings
desired_columns_order = [
    # General player info
    'Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born',
    
    # Playing time 
    'Playing Time MP', 'Playing Time Starts', 'Playing Time Min', 'Playing Time 90s',

    # Performance
    'Performance Gls', 'Performance Ast', 'Performance G+A', 'Performance G-PK',
    'Performance PK', 'Performance PKatt', 'Performance CrdY', 'Performance CrdR',
    'Performance 2CrdY', 'Performance Fls', 'Performance Fld', 'Performance Off', 'Performance Crs',
    'Performance Int', 'Performance OG', 'Performance Recov',

    # Expected performance
    'Expected xG', 'Expected npxG', 'Expected xAG', 'Expected npxG+xAG', 'Expected xAG', 'Expected xA', 'Expected A-xAG',

    # Expected performance ratios
    'Expected npxG/Sh', 'Expected G-xG', 'Expected np:G-xG',
    
    # Progression statistics
    'Progression PrgC', 'Progression PrgP', 'Progression PrgR',
    
    # Per 90 minutes performance
    'Per 90 Minutes Gls', 'Per 90 Minutes Ast', 'Per 90 Minutes G+A', 'Per 90 Minutes G-PK',
    'Per 90 Minutes G+A-PK', 'Per 90 Minutes xG', 'Per 90 Minutes xAG', 'Per 90 Minutes xG+xAG',
    'Per 90 Minutes npxG', 'Per 90 Minutes npxG+xAG',
    
    # Additional playing time info
    '90s',
    
    # Defensive statistics
    'Tackles Tkl', 'Tackles TklW', 'Tackles Def 3rd', 'Tackles Mid 3rd', 'Tackles Att 3rd',
    'Challenges Tkl', 'Challenges Att', 'Challenges Tkl%', 'Challenges Lost', 'Blocks Blocks',
    'Blocks Sh', 'Blocks Pass', 'Int', 'Tkl+Int', 'Clr', 'Err',
    
    # Standard performance statistics
    'Standard Gls', 'Standard Sh', 'Standard SoT', 'Standard SoT%', 'Standard Sh/90',
    'Standard SoT/90', 'Standard G/Sh', 'Standard G/SoT', 'Standard Dist', 'Standard FK',
    'Standard PK', 'Standard PKatt',
    
    
    # Goalkeeping performance
    'Performance GA', 'Performance GA90', 'Performance SoTA', 'Performance Saves',
    'Performance Save%', 'Performance W', 'Performance D', 'Performance L', 'Performance CS',
    'Performance CS%',
    
    # Penalty kicks
    'Penalty Kicks PKatt', 'Penalty Kicks PKA', 'Penalty Kicks PKsv', 'Penalty Kicks PKm',
    'Penalty Kicks Save%',
    
    # Scoring and chance creation
    'SCA SCA', 'SCA SCA90', 'SCA Types PassLive', 'SCA Types PassDead', 'SCA Types TO',
    'SCA Types Sh', 'SCA Types Fld', 'SCA Types Def', 'GCA GCA', 'GCA GCA90',
    'GCA Types PassLive', 'GCA Types PassDead', 'GCA Types TO', 'GCA Types Sh', 'GCA Types Fld',
    'GCA Types Def',
        
    # Aerial duels
    'Aerial Duels Won', 'Aerial Duels Lost', 'Aerial Duels Won%',
    
    # Passing statistics
    'Total Cmp', 'Total Att', 'Total Cmp%', 'Total TotDist', 'Total PrgDist',
    
    # Other statistics
    'KP', '1/3', 'PPA', 'CrsPA', 'PrgP'
]

# Select columns in the desired order
merged_df = merged_df[desired_columns_order]

# Delete unnecessary row (there are many  "Missing Value")
merged_df = merged_df[merged_df['Player'] != 'Blanco']
merged_df = merged_df[merged_df['Player'] != 'Hugo Guillamón']
merged_df = merged_df[merged_df['Player'] != 0]


In [18]:
# Delete duplicate columns from the DataFrame
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

# Delete duplicates respect to Rk, Player Squad
merged_df = merged_df.drop_duplicates(subset=['Rk', 'Player','Squad'])

In [19]:
# Convert column selected in int type
int_columns = [
    'Rk', 'Age', 'Born', 'Playing Time MP', 'Playing Time Starts', 'Performance Gls', 
    'Performance Ast', 'Performance G+A', 'Performance G-PK', 'Performance PK', 'Performance PKatt', 
    'Performance CrdY', 'Performance CrdR', 'Performance 2CrdY', 'Performance Fls', 'Performance Fld',
    'Performance Off', 'Performance Crs', 'Performance Int', 'Performance OG', 'Performance Recov', 
    'Progression PrgC', 'Progression PrgP', 'Progression PrgR', 'Tackles Tkl', 'Tackles TklW', 
    'Tackles Def 3rd', 'Tackles Mid 3rd', 'Tackles Att 3rd', 'Challenges Tkl', 'Challenges Att', 
    'Challenges Lost', 'Blocks Blocks', 'Blocks Sh', 'Blocks Pass', 'Int', 'Tkl+Int', 'Clr', 'Err',
    'Standard Gls', 'Standard Sh', 'Standard SoT', 'Standard FK', 'Standard PK', 'Standard PKatt', 
    'Performance GA', 'Performance SoTA', 'Performance Saves', 'Performance W', 'Performance D', 
    'Performance L', 'Performance CS', 'Penalty Kicks PKatt', 'Penalty Kicks PKA', 'Penalty Kicks PKsv', 
    'Penalty Kicks PKm', 'SCA SCA', 'SCA Types PassLive', 'SCA Types PassDead', 'SCA Types TO', 'SCA Types Sh', 
    'SCA Types Fld', 'SCA Types Def', 'GCA GCA', 'GCA Types PassLive', 'GCA Types PassDead', 'GCA Types TO', 
    'GCA Types Sh', 'GCA Types Fld', 'GCA Types Def', 'Aerial Duels Won', 'Aerial Duels Lost', 'Total Cmp', 'Total Att', 
    'Total TotDist', 'Total PrgDist', 'KP', '1/3', 'PPA', 'CrsPA', 'PrgP']

merged_df[int_columns] = merged_df[int_columns].astype(int)

In [20]:
# Convert column selected in float type

float_columns = [
    'Playing Time 90s', 'Expected xG', 'Expected npxG','Expected xAG',
    'Expected xA', 'Expected A-xAG','Expected npxG/Sh', 'Expected G-xG', 'Expected np:G-xG',
    'Per 90 Minutes Gls', 'Per 90 Minutes Ast', 'Per 90 Minutes G+A',
    'Per 90 Minutes G-PK', 'Per 90 Minutes G+A-PK', 'Per 90 Minutes xG',
    'Per 90 Minutes xAG', 'Per 90 Minutes xG+xAG', 'Per 90 Minutes npxG',
    'Per 90 Minutes npxG+xAG', '90s', 'Challenges Tkl%', 'Standard SoT%','Standard Sh/90', 'Standard SoT/90', 'Standard G/Sh', 'Standard G/SoT',
    'Standard Dist', 'Performance GA90', 'Performance Save%', 'Performance CS%',
    'Penalty Kicks Save%', 'SCA SCA90', 'GCA GCA90', 'Aerial Duels Won%',
    'Total Cmp%'
]

merged_df[float_columns] = merged_df[float_columns].astype(float)

In [21]:
# SAVE raw data complete (checkpoint)
merged_df.to_csv('football-player-stats-2023-24-COMPLETE.csv', index=False)
merged_df.dtypes

Rk         int32
Player    object
Nation    object
Pos       object
Squad     object
           ...  
KP         int32
1/3        int32
PPA        int32
CrsPA      int32
PrgP       int32
Length: 123, dtype: object

##### Final features after data cleaning
1. **Rk**: Index or rank of the player in the list.
2. **Player**: Full name of the player.
3. **Nation**: Player's country of origin.
4. **Pos**: The position in which the player plays, e.g., "FW" for forward.
5. **Squad**: The team the player belongs to.
6. **Comp**: The competition in which the player participated.
7. **Age**: The player's age.
8. **Born**: The player's date of birth.
9. **Playing Time MP**: The total number of matches the player participated in.
10. **Playing Time Starts**: The number of matches the player started.
11. **Playing Time Min**: The total minutes played by the player.
12. **Playing Time 90s**: The number of 90-minute periods played by the player.
13. **Performance Gls**: The number of goals scored by the player.
14. **Performance Ast**: The number of assists provided by the player.
15. **Performance G+A**: The sum of goals and assists by the player.
16. **Performance G-PK**: The number of goals scored excluding penalty kicks.
17. **Performance PK**: The number of penalty kicks scored by the player.
18. **Performance PKatt**: The number of penalty kicks attempted by the player.
19. **Performance CrdY**: The number of yellow cards received by the player.
20. **Performance CrdR**: The number of red cards received by the player.
21. **Performance 2CrdY**: The number of second yellow cards (subsequent yellow cards).
22. **Performance Fls**: The number of fouls committed by the player.
23. **Performance Fld**: The number of fouls suffered by the player.
24. **Performance Off**: The number of offside offenses committed by the player.
25. **Performance Crs**: The number of crosses executed by the player.
26. **Performance Int**: The number of interceptions made by the player.
27. **Performance OG**: The number of own goals scored by the player.
28. **Performance Recov**: The number of ball recoveries made by the player.
29. **Expected xG**: Expected Goals, i.e., the number of expected goals based on statistics.
30. **Expected npxG**: Non-Penalty Expected Goals, expected goals excluding penalties.
31. **Expected xAG**: Expected Assisted Goals, expected assists based on statistics.
32. **Expected npxG+xAG**: The sum of Non-Penalty Expected Goals and Expected Assisted Goals.
33. **Expected xA**: Expected Assists, expected assists based on statistics.
34. **Expected A-xAG**: The difference between Expected Assists and Expected Assisted Goals.
35. **Expected npxG/Sh**: Non-Penalty Expected Goals per shot attempted.
36. **Expected G-xG**: The difference between the actual number of goals and Expected Goals.
37. **Expected np:G-xG**: The difference between the actual number of non-penalty goals and Expected Goals.
38. **Progression PrgC**: The number of progressive ball carries. This represents how many times a player has carried the ball forward through dribbling or advancing with the ball after possession. It indicates the player's ability to bypass opponents and move towards the opponent's area.
39. **Progression PrgP**: The number of progressive passes made by the player. This represents how many passes the player has made that have advanced the ball towards the opponent's area. These passes can be long or vertical, contributing to the team's attack construction.
40. **Progression PrgR**: The number of progressive passes received by the player. This represents how many passes the player has received from teammates that have advanced the ball towards the opponent's area.
41. **Per 90 Minutes Gls**: Number of goals per 90 minutes played.
42. **Per 90 Minutes Ast**: Number of assists per 90 minutes played.
43. **Per 90 Minutes G+A**: Sum of goals and assists per 90 minutes played.
44. **Per 90 Minutes G-PK**: Number of goals excluding penalty kicks per 90 minutes played.
45. **Per 90 Minutes G+A-PK**: Sum of goals and assists excluding penalty kicks per 90 minutes played.
46. **Per 90 Minutes xG**: Expected Goals per 90 minutes played.
47. **Per 90 Minutes xAG**: Expected Assisted Goals per 90 minutes played.
48. **Per 90 Minutes xG+xAG**: Sum of Expected Goals and Expected Assisted Goals per 90 minutes played.
49. **Per 90 Minutes npxG**: Non-Penalty Expected Goals per 90 minutes played.
50. **Per 90 Minutes npxG+xAG**: Sum of Non-Penalty Expected Goals and Expected Assisted Goals per 90 minutes played.
52. **Tackles Tkl**: The number of tackle attempts made by the player.
53. **Tackles TklW**: The number of tackles won by the player.
54. **Tackles Def 3rd**: The number of tackles made in the defensive third. Indicates how many challenges the player has won in the area closest to their own goal, suggesting their ability to defend effectively and regain possession in the most critical area of the field.
55. **Tackles Mid 3rd**: The number of tackles made in the central third of the field. Refers to challenges won by the player in the middle part of the field, where ball recovery can impact the transition between defense and attack.
56. **Tackles Att 3rd**: The number of tackles made in the attacking third. Represents challenges won by the player in the area closest to the opponent's goal, indicating their involvement in ball recovery during the attacking and high-press phases.
58. **Challenges Att**: The total number of challenges faced by the player.
59. **Challenges Tkl%**: Percentage of challenges won compared to total challenges.
60. **Challenges Lost**: The number of challenges lost by the player.
61. **Blocks Blocks**: The number of times the player has blocked the ball.
62. **Blocks Sh**: The number of opponent shots blocked by the player.
63. **Blocks Pass**: The number of opponent passes blocked by the player.
64. **Int**: The number of interceptions made by the player.
65. **Tkl+Int**: The sum of tackle attempts and interceptions made by the player.
66. **Clr**: The number of balls cleared by the player.
67. **Err**: The number of errors committed by the player that led to opponent shots.
69. **Standard Gls**: The number of standard goals scored by the player.
70. **Standard Sh**: The total number of standard shots attempted by the player.
71. **Standard SoT**: The number of standard shots on target made by the player.
72. **Standard SoT%**: Percentage of standard shots on target compared to total standard shots.
73. **Standard Sh/90**: Number of standard shots per 90 minutes played.
74. **Standard SoT/90**: Number of standard shots on target per 90 minutes played.
75. **Standard G/Sh**: Number of goals scored per standard shot attempted by the player.
76. **Standard G/SoT**: Number of goals scored per standard shot on target made by the player.
77. **Standard Dist**: The average distance of shots attempted by the player.
78. **Standard FK**: The number of free-kick shots attempted by the player.
79. **Standard PK**: The number of penalty kicks scored by the player.
80. **Standard PKatt**: The number of penalty kicks attempted by the player.
81. **Performance GA**: The number of goals conceded by the team when the player is on the field. (GK)
82. **Performance GA90**: The number of goals conceded per 90 minutes played by the player. (GK)
83. **Performance SoTA**: The number of opponent shots on target faced by the team when the player is on the field. (GK)
84. **Performance Saves**: The number of shots saved by the goalkeeper when the player is on the field. (GK)
85. **Performance Save%**: Percentage of shots saved compared to shots faced. (GK)
86. **Performance W**: The number of matches won by the team when the player is on the field. (GK)
87. **Performance D**: The number of matches drawn by the team when the player is on the field. (GK)
88. **Performance L**: The number of matches lost by the team when the player is on the field. (GK)
89. **Performance CS**: The number of clean sheets (matches without conceding goals) achieved by the team when the player is on the field. (GK)
90. **Performance CS%**: Percentage of matches in which the team does not concede goals when the player is on the field. (GK)
91. **Penalty Kicks PKatt**: The number of penalty kicks attempted by the team when the player is on the field.
92. **Penalty Kicks PKA**: The number of penalty kicks conceded by the team when the player is on the field.
93. **Penalty Kicks PKsv**: The number of penalty kicks saved by the team when the player is on the field.
94. **Penalty Kicks PKm**: The number of penalty kicks missed by the team when the player is on the field.
95. **SCA SCA**: The total number of "Shot-Creating Actions" performed by the player. Shot-Creating Actions are a statistical metric used in soccer to measure a player's involvement in actions that directly lead to a shot attempt by their team.
96. **SCA SCA90**: The number of "Shot-Creating Actions" per 90 minutes played by the player.
97. **SCA Types PassLive**: Number of "Shot-Creating Actions" through live ball passes.
98. **SCA Types PassDead**: Number of  "Shot-Creating Actions" through dead ball passes.
99. **SCA Types TO**: Number of "Shot-Creating Actions" through successful dribbles (take-ons).
100. **SCA Types Sh**: Number of "Shot-Creating Actions" through shots.
101. **SCA Types Fld**: Number of "Shot-Creating Actions" through fouls drawn.
102. **SCA Types Def**: Number of "Shot-Creating Actions" through defensive actions.
103. **GCA GCA**: The total number of "Goal-Creating Actions" performed by the player. Goal-Creating Actions are a statistical metric in soccer that measures a player's involvement in actions that directly lead to a goal scored by their team.
104. **GCA GCA90**: The number of "Goal-Creating Actions" per 90 minutes played by the player.
105. **GCA Types PassLive**: Number of "Goal-Creating Actions" through live ball passes.
106. **GCA Types PassDead**: Number of "Goal-Creating Actions" through dead ball passes.
107. **GCA Types TO**: Number of "Goal-Creating Actions" through successful dribbles (take-ons).
108. **GCA Types Sh**: Number of "Goal-Creating Actions" through shots.
109. **GCA Types Fld**: Number of "Goal-Creating Actions" through fouls drawn.
110. **GCA Types Def**: Number of "Goal-Creating Actions" through defensive actions.
111. **Aerial Duels Won**: The number of aerial duels won by the player.
112. **Aerial Duels Lost**: The number of aerial duels lost by the player.
113. **Aerial Duels Won%**: Percentage of aerial duels won compared to total aerial duels.
114. **Total Cmp**: The total number of passes completed by the player.
115. **Total Att**: The total number of passes attempted by the player.
116. **Total Cmp%**: Percentage of passes completed compared to total passes attempted.
117. **Total TotDist**: The total distance covered by completed passes by the player. This metric indicates the total distance in meters or yards covered by the ball through all the passes completed by the player during the match.
118. **Total PrgDist**: The total distance covered by completed passes that advance towards the opponent's goal. This metric represents the total distance in meters or yards covered by passes completed that advance towards the opponent's goal, contributing to the attack progression.
119. **KP**: The number of "Key Passes" (key passes) made by the player.
120. **1/3**: The number of passes completed in the final third of the opponent's field.
121. **PPA**: The number of passes completed in the opponent's penalty area.
122. **CrsPA**: The number of crosses completed in the opponent's penalty area.


### Feature Selection
After cleaning the data, features were selected in order to identify and retain only the most relevant attributes. This was achieved by calculating and comparing the variances between the features in the dataset. Features with a variance below a predefined threshold were omitted, resulting in a more focused and efficient dataset. In addition, other columns that in context may be unnecessary for the construction of the recommendation system are eliminated.

In [22]:
# List of columns to be excluded from the variance calculation
excluded_columns = ['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born'] 

# Convert all columns except excluded ones to numeric, coerce errors to NaN
numeric_df = merged_df.drop(columns=excluded_columns).apply(pd.to_numeric, errors='coerce')

# Calculates the variance of the numeric columns, excluding the specified columns
variances = numeric_df.var()

# Specifies the upper limit for the variance
variance_threshold = 0.8

# Selects features with variance above the threshold
selected_features = variances[variances > variance_threshold].index

# Display the selected features
selected_features


Index(['Playing Time MP', 'Playing Time Starts', 'Playing Time Min',
       'Playing Time 90s', 'Performance Gls', 'Performance Ast',
       'Performance G+A', 'Performance G-PK', 'Performance CrdY',
       'Performance Fls', 'Performance Fld', 'Performance Off',
       'Performance Crs', 'Performance Int', 'Performance Recov',
       'Expected xG', 'Expected npxG', 'Expected xAG', 'Expected npxG+xAG',
       'Expected xA', 'Expected A-xAG', 'Expected G-xG', 'Expected np:G-xG',
       'Progression PrgC', 'Progression PrgP', 'Progression PrgR', '90s',
       'Tackles Tkl', 'Tackles TklW', 'Tackles Def 3rd', 'Tackles Mid 3rd',
       'Tackles Att 3rd', 'Challenges Tkl', 'Challenges Att',
       'Challenges Tkl%', 'Challenges Lost', 'Blocks Blocks', 'Blocks Sh',
       'Blocks Pass', 'Int', 'Tkl+Int', 'Clr', 'Standard Gls', 'Standard Sh',
       'Standard SoT', 'Standard SoT%', 'Standard Sh/90', 'Standard Dist',
       'Standard FK', 'Performance GA', 'Performance GA90', 'Performance SoTA

In [23]:
print(len(selected_features))
selected_features

84


Index(['Playing Time MP', 'Playing Time Starts', 'Playing Time Min',
       'Playing Time 90s', 'Performance Gls', 'Performance Ast',
       'Performance G+A', 'Performance G-PK', 'Performance CrdY',
       'Performance Fls', 'Performance Fld', 'Performance Off',
       'Performance Crs', 'Performance Int', 'Performance Recov',
       'Expected xG', 'Expected npxG', 'Expected xAG', 'Expected npxG+xAG',
       'Expected xA', 'Expected A-xAG', 'Expected G-xG', 'Expected np:G-xG',
       'Progression PrgC', 'Progression PrgP', 'Progression PrgR', '90s',
       'Tackles Tkl', 'Tackles TklW', 'Tackles Def 3rd', 'Tackles Mid 3rd',
       'Tackles Att 3rd', 'Challenges Tkl', 'Challenges Att',
       'Challenges Tkl%', 'Challenges Lost', 'Blocks Blocks', 'Blocks Sh',
       'Blocks Pass', 'Int', 'Tkl+Int', 'Clr', 'Standard Gls', 'Standard Sh',
       'Standard SoT', 'Standard SoT%', 'Standard Sh/90', 'Standard Dist',
       'Standard FK', 'Performance GA', 'Performance GA90', 'Performance SoTA

In [24]:
# Selected features need
selected_features_need = ['Performance GA', 'Performance GA90', 'Performance SoTA', 'Performance Saves',
    'Performance W', 'Performance D', 'Performance L', 'Performance CS', 'Expected xG']

selected_features.union(selected_features_need)

selected_df = merged_df[excluded_columns + list(selected_features)]

# Delete duplicate columns from the DataFrame
selected_df = selected_df.loc[:, ~selected_df.columns.duplicated()]

In [25]:
# Delete columns not affected

columns_to_drop = ['Performance CrdY','90s','Challenges Tkl', 'Performance W', 'Performance D', 'Performance L','SCA Types PassDead', 'SCA Types TO',
       'SCA Types Sh', 'SCA Types Fld', 'Playing Time Starts', 'Playing Time Min', 'Playing Time 90s','Tkl+Int', 'Standard Gls', 'SCA SCA90',
       'Performance Int','Performance Off','SCA Types PassLive','GCA Types PassLive']

selected_df = selected_df.drop(columns=columns_to_drop)


In [26]:
# checkpoint
selected_df

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,Playing Time MP,Performance Gls,...,Total Cmp,Total Att,Total Cmp%,Total TotDist,Total PrgDist,KP,1/3,PPA,CrsPA,PrgP
0,1,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,24,2000,0,0,...,450,581,77.5,7402,2789,7,25,13,2,43
1,2,Brenden Aaronson,us USA,"MF,FW",Union Berlin,de Bundesliga,24,2000,0,2,...,365,472,77.3,4890,1506,22,30,14,3,56
2,3,Paxten Aaronson,us USA,MF,Eint Frankfurt,de Bundesliga,21,2003,0,0,...,41,50,82.0,576,71,1,4,2,0,5
3,4,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,18,2006,0,0,...,1,1,100.0,8,0,0,0,0,0,0
4,5,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,37,1987,0,4,...,1552,1836,84.5,29618,9672,8,129,3,0,137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2847,2848,Lovro Zvonarek,hr CRO,"FW,MF",Bayern Munich,de Bundesliga,19,2005,0,1,...,50,60,83.3,766,123,2,5,1,0,7
2848,2849,Martin Ødegaard,no NOR,MF,Arsenal,eng Premier League,26,1998,0,8,...,1692,2006,84.3,25780,7898,102,160,130,13,344
2849,2850,Milan Đurić,ba BIH,FW,Hellas Verona,it Serie A,34,1990,0,5,...,181,334,54.2,2116,392,16,18,3,0,15
2850,2851,Milan Đurić,ba BIH,FW,Monza,it Serie A,34,1990,0,4,...,140,272,51.5,1700,251,5,3,2,0,8


### Data Aggregation and Transformation
The next step involves the aggregation of data according to individual players. Using aggregation functions such as sum and max, player-specific statistics were consolidated into complete player profiles without hypothetical repetition as players were duplicated due to the hypothetical change of team in the same year. The dataset was then meticulously sorted, ensuring an orderly representation to facilitate analysis.
The "Nation" and "Competion" columns were appropriately 'transformed' to enhanced the dataset's uniformity and prepared it for further analysis.

In [27]:
# Dictionary for mapping values to be replaced
replacement_dict = {
    "it Serie A": "Serie A",
    "fr Ligue 1": "Ligue 1",
    "de Bundesliga": "Bundesliga",
    "eng Premier League": "Premier League",
    "es La Liga": "La Liga"
}

# Apply the replacement using the replace method
selected_df['Comp'] = selected_df['Comp'].replace(replacement_dict)

# Remove the first three characters from the 'Nation' column
selected_df['Nation'] = selected_df['Nation'].str[3:]


In [28]:
# Aggregation

aggregation_functions = {
    "Rk": "first",
    "Nation": "first",
    "Pos": "first",
    "Squad": lambda x: ", ".join(x),
    "Comp": lambda x: ", ".join(x) if len(set(x)) > 1 else x.iloc[0],
    "Age": "first",
    "Playing Time MP": "sum",
    "Performance Gls": "sum",
    "Performance Ast": "sum",
    "Performance G+A": "sum",
    "Performance G-PK": "sum",
    "Performance Fls": "sum",
    "Performance Fld": "sum",
    "Performance Crs": "sum",
    "Performance Recov": "sum",
    "Expected xG": "max",
    "Expected npxG": "max",
    "Expected xAG": "max",
    "Expected xA": "max",
    "Expected A-xAG": "max",
    "Expected G-xG": "max",
    "Expected np:G-xG": "max",
    "Progression PrgC": "sum",
    "Progression PrgP": "sum",
    "Progression PrgR": "sum",
    "Tackles Tkl": "sum",
    "Tackles TklW": "sum",
    "Tackles Def 3rd": "sum",
    "Tackles Mid 3rd": "sum",
    "Tackles Att 3rd": "sum",
    "Challenges Att": "sum",
    "Challenges Tkl%": "sum",
    "Challenges Lost": "sum",
    "Blocks Blocks": "sum",
    "Blocks Sh": "sum",
    "Blocks Pass": "sum",
    "Int": "sum",
    "Clr": "sum",
    "Standard Sh": "sum",
    "Standard SoT": "sum",
    "Standard SoT%": "sum",
    "Standard Sh/90": "sum",
    "Standard Dist": "max",
    "Standard FK": "sum",
    "Performance GA": "sum",
    "Performance SoTA": "sum",
    "Performance Saves": "sum",
    "Performance Save%": "sum",
    "Performance CS": "sum",
    "Performance CS%": "sum",
    "Penalty Kicks PKatt": "sum",
    "Penalty Kicks Save%": "sum",
    "SCA SCA": "sum",
    "GCA GCA": "sum",
    "Aerial Duels Won": "sum",
    "Aerial Duels Lost": "sum",
    "Aerial Duels Won%": "sum",
    "Total Cmp": "sum",
    "Total Att": "sum",
    "Total Cmp%": "sum",
    "Total TotDist": "sum",
    "Total PrgDist": "sum",
    "KP": "sum",
    "1/3": "sum",
    "PPA": "sum",
    "CrsPA": "sum",
    "PrgP": "sum"
}

# Group the DataFrame by 'Player' and apply aggregation functions
aggregated_df = selected_df.groupby(['Player', 'Born']).agg(aggregation_functions).reset_index()


In [29]:
# Reorder row
aggregated_df['Rk'] = aggregated_df['Rk'].astype(int) 

# Rearranges the DataFrame according to the column "Rk"
aggregated_df = aggregated_df.sort_values(by='Rk', ascending=True)

aggregated_df['Rk'] = range(len(aggregated_df))

# Reset DataFrame index
aggregated_df = aggregated_df.reset_index(drop=True)

In [30]:
# checkpoint
aggregated_df

,Player,Born,Rk,Nation,Pos,Squad,Comp,Age,Playing Time MP,Performance Gls,...,Total Cmp,Total Att,Total Cmp%,Total TotDist,Total PrgDist,KP,1/3,PPA,CrsPA,PrgP
0,Max Aarons,2000,0,ENG,DF,Bournemouth,Premier League,24,0,0,...,450,581,77.5,7402,2789,7,25,13,2,43
1,Brenden Aaronson,2000,1,USA,"MF,FW",Union Berlin,Bundesliga,24,0,2,...,365,472,77.3,4890,1506,22,30,14,3,56
2,Paxten Aaronson,2003,2,USA,MF,Eint Frankfurt,Bundesliga,21,0,0,...,41,50,82.0,576,71,1,4,2,0,5
3,Keyliane Abdallah,2006,3,FRA,FW,Marseille,Ligue 1,18,0,0,...,1,1,100.0,8,0,0,0,0,0,0
4,Yunis Abdelhamid,1987,4,MAR,DF,Reims,Ligue 1,37,0,4,...,1552,1836,84.5,29618,9672,8,129,3,0,137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704,Szymon Żurkowski,1997,2704,POL,MF,Empoli,Serie A,27,0,4,...,193,245,78.8,2886,649,4,19,5,1,20
2705,Lovro Zvonarek,2005,2705,CRO,"FW,MF",Bayern Munich,Bundesliga,19,0,1,...,50,60,83.3,766,123,2,5,1,0,7
2706,Martin Ødegaard,1998,2706,NOR,MF,Arsenal,Premier League,26,0,8,...,1692,2006,84.3,25780,7898,102,160,130,13,344
2707,Milan Đurić,1990,2707,BIH,FW,"Hellas Verona, Monza",Serie A,34,0,9,...,321,606,105.7,3816,643,21,21,5,0,23


In [31]:
# Rearranges the DataFrame according to the specified columns

ordered_columns = [
    'Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born',
    'Playing Time MP', 'Performance Gls', 'Performance Ast',
    'Performance G+A', 'Performance G-PK', 'Performance Fls',
    'Performance Fld', 'Performance Crs', 'Performance Recov',
    'Expected xG', 'Expected npxG', 'Expected xAG', 'Expected xA',
    'Expected A-xAG', 'Expected G-xG', 'Expected np:G-xG',
    'Progression PrgC', 'Progression PrgP', 'Progression PrgR',
    'Tackles Tkl', 'Tackles TklW', 'Tackles Def 3rd', 'Tackles Mid 3rd',
    'Tackles Att 3rd', 'Challenges Att', 'Challenges Tkl%',
    'Challenges Lost', 'Blocks Blocks', 'Blocks Sh', 'Blocks Pass', 'Int',
    'Clr', 'Standard Sh', 'Standard SoT', 'Standard SoT%', 'Standard Sh/90',
    'Standard Dist', 'Standard FK', 'Performance GA', 'Performance SoTA',
    'Performance Saves', 'Performance Save%', 'Performance CS',
    'Performance CS%', 'Penalty Kicks PKatt', 'Penalty Kicks Save%',
    'SCA SCA', 'GCA GCA', 'Aerial Duels Won', 'Aerial Duels Lost',
    'Aerial Duels Won%', 'Total Cmp', 'Total Att', 'Total Cmp%',
    'Total TotDist', 'Total PrgDist', 'KP', '1/3', 'PPA', 'CrsPA', 'PrgP'
]

aggregated_df = aggregated_df[ordered_columns]

In [32]:
# test same name player but different player
aggregated_df[aggregated_df['Player'] == 'Lamine Yamal']

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,Playing Time MP,Performance Gls,...,Total Cmp,Total Att,Total Cmp%,Total TotDist,Total PrgDist,KP,1/3,PPA,CrsPA,PrgP
2654,2654,Lamine Yamal,ESP,FW,Barcelona,La Liga,17,2007,0,5,...,903,1132,79.8,12868,3009,38,39,52,16,80


In [33]:
# DATASET PREPROCESSED
aggregated_df.to_csv('football-player-stats-2023-24.csv', index=False)
aggregated_df

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,Playing Time MP,Performance Gls,...,Total Cmp,Total Att,Total Cmp%,Total TotDist,Total PrgDist,KP,1/3,PPA,CrsPA,PrgP
0,0,Max Aarons,ENG,DF,Bournemouth,Premier League,24,2000,0,0,...,450,581,77.5,7402,2789,7,25,13,2,43
1,1,Brenden Aaronson,USA,"MF,FW",Union Berlin,Bundesliga,24,2000,0,2,...,365,472,77.3,4890,1506,22,30,14,3,56
2,2,Paxten Aaronson,USA,MF,Eint Frankfurt,Bundesliga,21,2003,0,0,...,41,50,82.0,576,71,1,4,2,0,5
3,3,Keyliane Abdallah,FRA,FW,Marseille,Ligue 1,18,2006,0,0,...,1,1,100.0,8,0,0,0,0,0,0
4,4,Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,37,1987,0,4,...,1552,1836,84.5,29618,9672,8,129,3,0,137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704,2704,Szymon Żurkowski,POL,MF,Empoli,Serie A,27,1997,0,4,...,193,245,78.8,2886,649,4,19,5,1,20
2705,2705,Lovro Zvonarek,CRO,"FW,MF",Bayern Munich,Bundesliga,19,2005,0,1,...,50,60,83.3,766,123,2,5,1,0,7
2706,2706,Martin Ødegaard,NOR,MF,Arsenal,Premier League,26,1998,0,8,...,1692,2006,84.3,25780,7898,102,160,130,13,344
2707,2707,Milan Đurić,BIH,FW,"Hellas Verona, Monza",Serie A,34,1990,0,9,...,321,606,105.7,3816,643,21,21,5,0,23


## Data Export on Apache Solr
After data processing, the final data set was imported into **Apache Solr**. Solr is commonly used for indexing and searching large datasets, which makes it a valuable tool for data retrieval and analysis. However, Apache Solr has no official Python documentation and libraries, particularly for version 3.10. The choice was made to use the Python SolrClient library for data import and retrieval. Here is the interaction between App-SolrClient-Apache Solr:

In [32]:
import json
from SolrClient import SolrClient
solr = SolrClient('http://localhost:8983/solr') # you must first run Apache Solr locally

In [33]:
# to dict data
docs = aggregated_df.to_dict(orient='records')

In [35]:
# indexing the JSON data in the Solr "FootballStatsCore" core
solr.index_json("FootballStatsCore", json.dumps(docs) )
solr.commit("FootballStatsCore", softCommit=True)